In [15]:
import re, string, unicodedata
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import pandas as pd
import xlsxwriter
from nltk.tokenize import sent_tokenize
from utils import constant 
import csv

In [16]:
def casefolding(str):
    new_str = str.lower()
    return new_str

def stopword(str):
    stop_words = set(stopwords.words('stopwordID.csv'))
    word_tokens = word_tokenize(str)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)


def remove_eng(str):
    stop_words = set(stopwords.words('english.csv')) 
    word_tokens = word_tokenize(str) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]  
    return ' '.join(filtered_sentence)

#remove sentence which contains only one word
def removeSentence(str): 
    word = str.split()
    wordCount = len(word)
    if(wordCount<=1):
        str = ''
    
    return str



def cleaning(str):
    #remove punctuations
    str = re.sub(r'[^\w]|_',' ',str)
    #remove digit from string
    str = re.sub("\S*\d\S*", "", str).strip()
    #removeHashtag
    str = re.sub('#[^\s]+','',str)
    #remove non-ascii
    str = unicodedata.normalize('NFKD', str).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #to lowercase
    str = str.lower()
    #Remove additional white spaces
    str = re.sub('[\s]+', ' ', str)
    
    return str
    
#slang word
def normalize_slang_word(str):
    text_list = str.split(' ')
    slang_words_raw = pd.read_csv('slang_word_list.csv', sep=',', header=None)
    slang_word_dict = {}
    
    for item in slang_words_raw.values:
        slang_word_dict[item[0]] = item[1]
        
        for index in range(len(text_list)):
            if text_list[index] in slang_word_dict.keys():
                text_list[index] = slang_word_dict[text_list[index]]
    
    return ' '.join(text_list) 

def normalize_bahasa_jawa(str):
    text_list = str.split(' ')
    jawa_raw = pd.read_csv('bahasa jawa.csv', sep=',', header=None)
    jawa_dict = {}
    
    for item in jawa_raw.values:
        jawa_dict[item[0]] = item[1]
        
        for index in range(len(text_list)):
            if text_list[index] in jawa_dict.keys():
                text_list[index] = jawa_dict[text_list[index]]
    
    return ' '.join(text_list) 

def remove_repeated_character(str):
    str = re.sub(r'(.)\1{2,}', r'\1', str)
    
    return str

def join_phrase(str):

    text_list = str.split(' ')

    negation_word = ['tidak','kurang','belum','cukup','sangat','bagian']
    for index in range(len(text_list)):
        for kt in negation_word:
            if text_list[index] == kt:
                if index < len(text_list) - 1:
                    text_list[index] = text_list[index] + '_' + text_list[index + 1]
                    text_list[index + 1] = ''
                else:
                    text_list[index] = ''
    return (' '.join(text_list))

def normalize_emoticon(str):
        text_list = str.split(' ')
        emoticon_list = constant.EMOTICON_LIST
        for index in range(len(text_list)):
            for key, value in emoticon_list:
                if text_list[index] in value:
                    text_list[index] = key
        return ' '.join(text_list)

def sentence_tokenization(s):
    sentences_list = sent_tokenize(s)
    
    return sentences_list

def split_sentences(str):
    kata = ["tapi","Tapi","Tetapi","namun","Namun","tetapi","saran"]
    word_tokens = word_tokenize(str) 
    sent = []
    sentences = []
    for w in word_tokens:
        if w in kata:
            sentences.append(' '.join(sent))
            sent = []
        else:
            sent.append(w)
    sentences.append(' '.join(sent))
    return sentences

def stemmingIndo(str):
    factory = StemmerFactory()
    #stemming bahasa indonesia
    stemmer = factory.create_stemmer()
    return stemmer.stem(str)


def preprocessing(str):
    str = normalize_slang_word(str)
#     str = normalize_emoticon(str)
#     str = cleaning(str) 
#     str = remove_repeated_character(str)
#     str = normalize_bahasa_jawa(str)
#     str = join_phrase(str)
#     str = ''.join(str)
#     str = str.replace('diterjemahkan',' ')
#     str = str.replace('google',' ')
#     str = str.replace('salah satu','')
#     str = str.replace('terima kasih','terima_kasih')
#     str = str.replace('rumah sakit','rumah_sakit')
#     str = str.replace('rawat inap','rawat_inap')
#     str = str.replace('panti rapih','panti_rapih')
    str = stopword(str)
#     str = remove_eng(str)
#     str = removeSentence(str)
    return str

In [30]:
kata =["RS rekomended letknya wa Bethesda adalah rs swasta tapi cukup_ekonomis dibanding rs swasta lainya oh ya pengalaman aku biar bpjs rs tetap memberikan mutu yg baik Tapi obat misal akan diberikan paten nah bagi yg anter berobat rs jangan kawatir susah cari makanan rs cafe jasmine toko roti jual jajan dlm rs atm bagi yg bth duit cash kalau luar rs atm bri rs jugaa dilengkapi kantin pintu barat masuk pengunjung nah yg lebih famous bakso nih sampingny bakso bethesda bakso enak harga lumayan premium seporsi rb kalau favorit ku bakso pak topi jual luar rs bethesda tepat seberang novotel hihihi"]
kt = ' '.join(kata)
benar = split_sentences(kt)
print(benar)

['RS rekomended letknya wa Bethesda adalah rs swasta', 'cukup_ekonomis dibanding rs swasta lainya oh ya pengalaman aku biar bpjs rs tetap memberikan mutu yg baik', 'obat misal akan diberikan paten nah bagi yg anter berobat rs jangan kawatir susah cari makanan rs cafe jasmine toko roti jual jajan dlm rs atm bagi yg bth duit cash kalau luar rs atm bri rs jugaa dilengkapi kantin pintu barat masuk pengunjung nah yg lebih famous bakso nih sampingny bakso bethesda bakso enak harga lumayan premium seporsi rb kalau favorit ku bakso pak topi jual luar rs bethesda tepat seberang novotel hihihi']


In [18]:
fo = pd.read_excel('E:\\kuliah\\Pengolahan Bahasa Alami\\Bismillah Skripsi Baru\\Tugas Akhir\\File Positif v4.xlsx') #read excel file
#fo = pd.read_csv('review JIH.csv',error_bad_lines=False)
review = fo['Review']
workbook = xlsxwriter.Workbook('E:\\kuliah\\Pengolahan Bahasa Alami\\Bismillah Skripsi Baru\\Tugas Akhir\\File Positif v4.xlsx')
worksheet = workbook.add_worksheet()
#type(review)
row = 0
col = 0

rowHeaders = ['Review']
worksheet.write_row(row, col,  tuple(rowHeaders))
        
for p in review:
    f = str(p)
#     st = sentence_tokenization(f)
    sf = preprocessing(f)
    rowValues = [sf]
    row += 1
    worksheet.write_row(row, col, tuple(rowValues))
   
workbook.close()

In [5]:
fo = pd.read_excel('E:\\kuliah\\Pengolahan Bahasa Alami\\Bismillah Skripsi Baru\\file skripsi\\kumpul.xlsx') #read excel file
#fo = pd.read_csv('review JIH.csv',error_bad_lines=False)
review = fo['Review']

#print(review)
workbook = xlsxwriter.Workbook('file skripsi\kumpul.xlsx')
worksheet = workbook.add_worksheet()
#type(review)
row = 0
col = 0

rowHeaders = ['Review']
worksheet.write_row(row, col,  tuple(rowHeaders))
        
for p in review:
    f = str(p)
    reviewrs = preprocessing(f)
    rowValues = [reviewrs]
    row += 1
    worksheet.write_row(row, col, tuple(rowValues))
   
workbook.close()

In [ ]:
fo = pd.read_excel('E:\\kuliah\Pengolahan Bahasa Alami\\Bismillah Skripsi Baru\\Data_Bersih\\Positif sw 2.xlsx') #read excel file
#fo = pd.read_csv('review JIH.csv',error_bad_lines=False)
review = fo['Review']
workbook = xlsxwriter.Workbook('E:\\kuliah\\Pengolahan Bahasa Alami\\Bismillah Skripsi Baru\\Data_Bersih\\Positif sw 2.xlsx')
worksheet = workbook.add_worksheet()
#type(review)
row = 0
col = 0

rowHeaders = ['Review']
worksheet.write_row(row, col,  tuple(rowHeaders))
        
for p in review:
    f = str(p)
    st = preprocessing(f)
    rowValues = [st]
    row += 1
    worksheet.write_row(row, col, tuple(rowValues))
   
workbook.close()